# Simulation and Optimization

We'll go through the entire simulation and optimization process in this tutorial. The general procedure on optimizing the portfolio is as follows:

1. Determine the asset classes (and thus the dataset used)
2. Simulate the returns
3. Run an optimization program on the simulated cube (tensor)

More details on each section will be covered.

## Getting Started

Please make sure you have the relevant packages installed. With the exception of `allopy`, most of the other packages can be installed from conda.

The procedure is as follows:

1. Copy and paste the following into a file called `env.yaml`. 

```
name: simop
channels:
  - defaults
  - conda-forge
  - anaconda
  - bashtage
  - danielbok
dependencies:
  - arch=4.8.1=py37hc5d7abb_0
  - attrs=19.1.0=py37_1
  - backcall=0.1.0=py37_0
  - blas=1.0=mkl
  - bleach=3.1.0=py37_0
  - ca-certificates=2019.1.23=0
  - certifi=2019.3.9=py37_0
  - colorama=0.4.1=py37_0
  - copulae=0.4.1=py37hfa6e2cd_0
  - cython=0.29.8=py37ha925a31_0
  - decorator=4.4.0=py37_1
  - defusedxml=0.6.0=py_0
  - entrypoints=0.3=py37_0
  - icc_rt=2019.0.0=h0cc432a_1
  - intel-openmp=2019.3=203
  - ipykernel=5.1.0=py37h39e3cac_0
  - ipython=7.5.0=py37h39e3cac_0
  - ipython_genutils=0.2.0=py37_0
  - jedi=0.13.3=py37_0
  - jinja2=2.10.1=py37_0
  - jsonschema=3.0.1=py37_0
  - jupyter_client=5.2.4=py37_0
  - jupyter_core=4.4.0=py37_0
  - libsodium=1.0.16=h9d3ae62_0
  - m2w64-gcc-libgfortran=5.3.0=6
  - m2w64-gcc-libs=5.3.0=7
  - m2w64-gcc-libs-core=5.3.0=7
  - m2w64-gmp=6.1.0=2
  - m2w64-libwinpthread-git=5.0.0.4634.697f757=2
  - markupsafe=1.1.1=py37he774522_0
  - mistune=0.8.4=py37he774522_0
  - mkl=2019.3=203
  - mkl_fft=1.0.12=py37h14836fe_0
  - mkl_random=1.0.2=py37h343c172_0
  - msys2-conda-epoch=20160418=1
  - muarch=0.0.7=py37
  - nbconvert=5.5.0=py_0
  - nbformat=4.4.0=py37_0
  - notebook=5.7.8=py37_0
  - numpy=1.16.4=py37h19fb1c0_0
  - numpy-base=1.16.4=py37hc3f5095_0
  - openssl=1.1.1c=he774522_1
  - pandas=0.24.2=py37ha925a31_0
  - pandoc=2.2.3.2=0
  - pandocfilters=1.4.2=py37_1
  - parso=0.4.0=py_0
  - patsy=0.5.1=py37_0
  - pickleshare=0.7.5=py37_0
  - pip=19.1.1=py37_0
  - prometheus_client=0.6.0=py37_0
  - prompt_toolkit=2.0.9=py37_0
  - pygments=2.4.0=py_0
  - pyrsistent=0.14.11=py37he774522_0
  - python=3.7.3=h8c8aaf0_1
  - python-dateutil=2.8.0=py37_0
  - pytz=2019.1=py_0
  - pywinpty=0.5.5=py37_1000
  - pyzmq=18.0.0=py37ha925a31_0
  - scipy=1.2.1=py37h29ff71c_0
  - send2trash=1.5.0=py37_0
  - setuptools=41.0.1=py37_0
  - six=1.12.0=py37_0
  - sqlite=3.28.0=he774522_0
  - statsmodels=0.9.0=py37h452e1ab_0
  - terminado=0.8.2=py37_0
  - testpath=0.4.2=py37_0
  - tornado=6.0.2=py37he774522_0
  - traitlets=4.3.2=py37_0
  - vc=14.1=h21ff451_3
  - vs2015_runtime=15.5.2=3
  - wcwidth=0.1.7=py37_0
  - webencodings=0.5.1=py37_1
  - wheel=0.33.4=py37_0
  - wincertstore=0.2=py37_0
  - winpty=0.4.3=4
  - zeromq=4.3.1=h33f27b4_3
```

2. Run `conda env create -f env.yaml -n my_env` in your terminal where you've saved this `env.yaml` file
3. Activate your environment via `conda activate my_env`
4. Get the `allopy` package from Daniel
5. Within the same, run `conda install allopy.***.tar.gz`

Now that you have the basic packages, we'll move on to the simulation.

## Simulation

For the simuluation The simulation follows the following procedure:

1. Load the **returns** dataset of the asset classes you want to simulate
2. For each asset class, specify the AR-GARCH model
3. Fit the AR-GARCH models with the **log-returns** data
4. Fit the standardized residuals of the AR-GARCH models to a Student Copula
5. Overwrite the correlation matrix of the Student Copula with the **log-returns correlation**
6. Simulate future returns using the AR-GARCH models with distributions "inversed" from the copula
7. Truncate then calibrate the first 2 moments (returns and vol) of the cube

### Exploring the Data

Let's start by loading and exploring the sample policy index dataset. These are the monthly returns for the 7 assets we will simulate. The data ranges from 01 Jan 1985 to 01 Oct 2017.

In [1]:
import numpy as np
from allopy.datasets import load_policy_index

returns = load_policy_index()
log_returns = (returns + 1).apply(np.log)
_, num_assets = log_returns.shape

log_returns.head()

,CASH,NB,EILB,DMEQ,EMEQ,PE,RE
DATE,,,,,,,
1985-01-01,0.004109,0.008796,0.028741,0.058068,-0.014185,0.067508,-0.000949
1985-01-02,0.024052,-0.002144,-0.052592,0.008257,-0.041216,0.007606,-0.018895
1985-01-03,-0.030244,0.006433,-0.000043,-0.013792,0.017271,0.028742,-0.014230
1985-01-04,0.002630,0.005479,0.020288,-0.083561,-0.005833,-0.016684,-0.022006
1985-01-05,0.007554,0.042372,0.046989,0.052969,-0.008937,0.071804,0.035114


In [2]:
log_returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 394 entries, 1985-01-01 to 2017-01-10
Data columns (total 7 columns):
CASH    394 non-null float64
NB      394 non-null float64
EILB    394 non-null float64
DMEQ    394 non-null float64
EMEQ    394 non-null float64
PE      394 non-null float64
RE      394 non-null float64
dtypes: float64(7)
memory usage: 24.6 KB


### Forming the Models

By right, we are supposed to form a distinct AR-GARCH model for each asset class. However, we'll take some short cuts by assuming every model is an AR(1)-GARCH(1,1)-Skew_T model with the exception of **CASH**. This exception is meant to serve as an example of how to change each model separately.

In [3]:
from muarch import MUArch, UArch

# The settings defined will set the default for all models. 
arch_models = MUArch(num_assets, 
                     mean='ARX', 
                     lags=1,
                     vol='GARCH',
                     p=1,
                     q=1,
                     power=2.0,
                     dist='skewt',
                     scale=100)

# You can specify each model separately. We'll use CASH as an example
# CASH is the model in the MUArch object
arch_models[0] = UArch(mean='CONSTANT', 
                       lags=0,
                       vol='CONSTANT',
                       dist='skewt',
                       scale=100)

### Fitting the Model

We can call the fit function and subsequently check the quality of each fit through the `.summary()` method.

In [4]:
arch_models.fit(log_returns)
arch_models.summary()

<class 'muarch.summary.SummaryList'>
"""
CASH

                     Constant Mean - Constant Variance Model Results                     
=========================================================================================
Dep. Variable:                                 y   R-squared:                      -0.000
Mean Model:                        Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                     Constant Variance   Log-Likelihood:               -734.606
Distribution:      Standardized Skew Student's t   AIC:                           1477.21
Method:                       Maximum Likelihood   BIC:                           1493.12
                                                   No. Observations:                  394
Date:                           Sat, Jul 13 2019   Df Residuals:                      390
Time:                                   11:10:15   Df Model:                            4
                                Mean Model                               
=========================================================================
                  coef    std err          t      P>|t|  95.0% Conf. Int.
-------------------------------------------------------------------------
mu         -4.6322e-03  8.148e-02 -5.685e-02      0.955 [ -0.164,  0.155]
                            Volatility Model                            
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
sigma2         2.6251      0.274      9.575  1.019e-21 [  2.088,  3.162]
                               Distribution                              
=========================================================================
                  coef    std err          t      P>|t|  95.0% Conf. Int.
-------------------------------------------------------------------------
nu              5.4964      1.387      3.964  7.368e-05 [  2.779,  8.214]
lambda     -6.7665e-04  6.797e-02 -9.955e-03      0.992 [ -0.134,  0.133]
=========================================================================

Covariance estimator: robust

****************************************************************************************************

NB

                                 AR - GARCH Model Results                                
=========================================================================================
Dep. Variable:                                 y   R-squared:                       0.009
Mean Model:                                   AR   Adj. R-squared:                  0.006
Vol Model:                                 GARCH   Log-Likelihood:               -651.331
Distribution:      Standardized Skew Student's t   AIC:                           1316.66
Method:                       Maximum Likelihood   BIC:                           1344.48
                                                   No. Observations:                  393
Date:                           Sat, Jul 13 2019   Df Residuals:                      386
Time:                                   11:10:15   Df Model:                            7
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
Const          0.1950  7.099e-02      2.746  6.029e-03  [5.582e-02,  0.334]
y[1]           0.0896  5.778e-02      1.550      0.121 [-2.368e-02,  0.203]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-------------------------------------------------------------------------

### Fitting the Copula

The next thing we need to do is to fit the copula with the standardized residuals of the arch models' fits. We can also check the fit of the copula with the `.summary()` method.

In [5]:
from copulae import TCopula

residuals = arch_models.residuals()  # defaults to standardized residuals
cop = TCopula(num_assets)
cop.fit(residuals)
cop.summary()

Student Copula Summary
================================================================================
Student Copula with 7 dimensions


Parameters
--------------------------------------------------------------------------------
 Degree of Freedom  : 23.06610779346201

 Correlation Matrix 
 1.000000  0.132348 -0.383801  0.089014  0.095759  0.069694  0.233250
 0.132348  1.000000  0.399801 -0.019929 -0.072426 -0.048833  0.178150
-0.383801  0.399801  1.000000  0.014783  0.037997  0.070883  0.015624
 0.089014 -0.019929  0.014783  1.000000  0.506281  0.699153  0.467578
 0.095759 -0.072426  0.037997  0.506281  1.000000  0.579079  0.317627
 0.069694 -0.048833  0.070883  0.699153  0.579079  1.000000  0.442582
 0.233250  0.178150  0.015624  0.467578  0.317627  0.442582  1.000000



                                  Fit Summary                                   
================================================================================
Log. Likelihood      : -388.7861311879695
Variance Estimate    : Not Implemented Yet
Method               : Maximum pseudo-likelihood
Data Points          : 393

Optimization Setup
--------------------------------------------------------------------------------
	bounds         : [(0.0, inf), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0)]
	options        : {'maxiter': 20000, 'ftol': 1e-06, 'iprint': 1, 'disp': False, 'eps': 1.5e-08}
	method         : SLSQP

Results
--------------------------------------------------------------------------------
	x              : [ 2.30661078e+01  1.32347651e-01 -3.83800649e-01  8.90135925e-02
  9.57587945e-02  6.96941173e-02  2.33250198e-01  3.99801367e-01
 -1.99293720e-02 -7.24255533e-02 -4.88326234e-02  1.78150082e-01
  1.47826773e-02  3.79967067e-02  7.08825763e-02  1.56243959e-02
  5.06280953e-01  6.99152789e-01  4.67577975e-01  5.79079326e-01
  3.17627052e-01  4.42581674e-01]
	fun            : -388.7861311879695
	jac            : [-0.00060254  0.01808758 -0.01784504 -0.0144117   0.02964953  0.01397969
 -0.02000509  0.02806928  0.00818166 -0.02256305 -0.01188027  0.02291548
 -0.02932363  0.0265004   0.02206662 -0.01734482  0.01431317 -0.03379152
 -0.00191373 -0.00712438  0.01340368  0.00355082]
	nit            : 36
	nfev           : 928
	njev           : 36
	status         : 0
	message        : Optimization terminated successfully.
	success        : True

All elliptical copula comes with a correlation matrix. In this case, we would like to overwrite the correlation matrix of the `TCopula` with the correlation matrix of the log returns. 

In [6]:
np.set_printoptions(linewidth=160)
cop.sigma

array([[ 1.        ,  0.13234765, -0.38380065,  0.08901359,  0.09575879,  0.06969412,  0.2332502 ],
       [ 0.13234765,  1.        ,  0.39980137, -0.01992937, -0.07242555, -0.04883262,  0.17815008],
       [-0.38380065,  0.39980137,  1.        ,  0.01478268,  0.03799671,  0.07088258,  0.0156244 ],
       [ 0.08901359, -0.01992937,  0.01478268,  1.        ,  0.50628095,  0.69915279,  0.46757797],
       [ 0.09575879, -0.07242555,  0.03799671,  0.50628095,  1.        ,  0.57907933,  0.31762705],
       [ 0.06969412, -0.04883262,  0.07088258,  0.69915279,  0.57907933,  1.        ,  0.44258167],
       [ 0.2332502 ,  0.17815008,  0.0156244 ,  0.46757797,  0.31762705,  0.44258167,  1.        ]])

In [7]:
# this is how you overwrite the correlation matrix

cop[:] = log_returns.corr()
cop.sigma

array([[ 1.        ,  0.13569432, -0.35933974,  0.10954112,  0.1281724 ,  0.09676939,  0.24841052],
       [ 0.13569432,  1.        ,  0.34423657, -0.01144135, -0.11482461, -0.06096223,  0.17947296],
       [-0.35933974,  0.34423657,  1.        ,  0.0869657 ,  0.05494202,  0.11039716,  0.03943829],
       [ 0.10954112, -0.01144135,  0.0869657 ,  1.        ,  0.59790511,  0.76230971,  0.49290886],
       [ 0.1281724 , -0.11482461,  0.05494202,  0.59790511,  1.        ,  0.67644088,  0.3457239 ],
       [ 0.09676939, -0.06096223,  0.11039716,  0.76230971,  0.67644088,  1.        ,  0.43281785],
       [ 0.24841052,  0.17947296,  0.03943829,  0.49290886,  0.3457239 ,  0.43281785,  1.        ]])

### Simulating Future returns

Now that we have the copula to generate a dependency structure for each of the univariate GARCH models, we can start simulating future returns. The simulated returns data will be a 3-dimensional cube with axis time, trials and asset class respectively.

For 10000 trials and 120 periods (10 years), this will usually take some time, so give it a while. For demonstration purposes, we'll be simulating a (120, 1000, 7) cube. 

**Note that the order is very important in the subsequent optimization procedure, so don't reshape it!**

In [8]:
simulated_log_returns = arch_models.simulate_mc(120, 1000, custom_dist=cop.random)
cube = np.exp(simulated_log_returns) - 1  # recover as returns, instead of log returns

### Removing Outliers

For each asset class, we define outliers as a data point that is more than 6 standard deviations away from the asset class's mean. We replace these outliers with the asset class mean. 

*As an aside, there probably is a better approach for multi-variate outlier detection and replacement*

In [9]:
from muarch.calibrate import truncate_outliers

cube = truncate_outliers(cube, 6)

### Calibrating the Moments of the Cube

After removing outliers in the cube, the next step is to calibrate the first 2 moments of the cube according to the forward looking assumptions that you have. The listing below shows the function to calculate the annualized mean and volatility for the data cube.

In [10]:
import pandas as pd


def show_moments(data):
    t, n, a = data.shape
    
    df = pd.DataFrame({
        'Asset': returns.columns,
        'Mean (%)': ((cube + 1).prod(0) ** 0.1).mean(0) - 1,
        'Vol (%)': ((cube + 1).reshape(t // 12, 12, n, a).prod(1) - 1).std(0).mean(0)
    })
    
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: round(100 * x, 4))
    return df

In [11]:
# before calibration

show_moments(cube)

,Asset,Mean (%),Vol (%)
0,CASH,-0.1069,5.0936
1,NB,2.6349,4.3602
2,EILB,9.5060,11.9566
3,DMEQ,4.1827,14.9285
4,EMEQ,14.3747,30.6739
5,PE,6.3590,19.0668
6,RE,2.0028,8.4100


For expedience, the calibration code is shown below.

```python
from scipy import optimize as opt

def calibrate_data(data, 
                   mean = None, 
                   sd = None, 
                   time_unit=12):
    data = data.copy()
    y, n, num_assets = data.shape
    y //= time_unit

    def set_target(target_values, typ, default):
        if target_values is None:
            return default, [0 if typ == 'mean' else 1] * num_assets

        best_guess = []
        target_values = np.asarray(target_values)
        assert num_assets == len(target_values) == len(
            default), "vector length must be equal to number of assets in data cube"
        for i, v in enumerate(target_values):
            if v is None:
                target_values[i] = default[i]
                best_guess.append(0 if typ == 'mean' else 1)
            else:
                best_guess.append(target_values[i] - default[i] if typ == 'mean' else default[i] / target_values[i])
        return target_values, best_guess

    d = (data + 1).prod(0)
    default_mean = (np.sign(d) * np.abs(d) ** (1 / y)).mean(0) - 1
    default_vol = ((data + 1).reshape(y, time_unit, n, num_assets).prod(1) - 1).std(1).mean(0)

    target_means, guess_mean = set_target(mean, 'mean', default_mean)
    target_vols, guess_vol = set_target(sd, 'vol', default_vol)

    sol = np.asarray([opt.root(
        fun=_asset_moments,
        x0=[gv, gm],
        args=(data[..., i], tv, tm, time_unit)
    ).x for i, tv, tm, gv, gm in zip(range(num_assets), target_vols, target_means, guess_vol, guess_mean)])

    for i in range(num_assets):
        if sol[i, 0] < 0 or False:
            # negative vol adjustments would alter the correlation between asset classes (flip signs)
            # in such instances, we fall back to using the a simple affine transform where
            # R' = (tv/cv) * r  # adjust vol first
            # R' = (tm - mean(R'))  # adjust mean

            # adjust vol
            tv = default_vol[i]
            cv = sd[i] if sd[i] is not None else tv
            data[..., i] *= tv / cv  # tv / cv

            # adjust mean
            tm, cm = target_means[i], data[..., i].mean()
            data[..., i] += (tm - cm)  # (tm - mean(R'))
        else:
            data[..., i] = data[..., i] * sol[i, 0] + sol[i, 1]
    return data


def _asset_moments(x: np.ndarray, asset: np.ndarray, t_vol: float, t_mean: float, time_unit: int):
    t, n = asset.shape  # time step (month), trials
    y = t // time_unit

    calibrated = asset * x[0] + x[1]

    d = (calibrated + 1).prod(0)
    mean = (np.sign(d) * np.abs(d) ** (1 / y)).mean() - t_mean - 1
    vol = ((calibrated.reshape((y, time_unit, n)) + 1).prod(1) - 1).std(1).mean(0) - t_vol

    return vol, mean
```

In [12]:
# calibration

from muarch.calibrate import calibrate_data

target_mean = [0, 0.03, 0.06, 0.08, 0.11, 0.12, 0.05]
target_vol =  [0.055, 0.073, 0.14, 0.09, 0.22, 0.18, 0.1]

cube = calibrate_data(cube, target_mean, target_vol)

show_moments(cube)

,Asset,Mean (%),Vol (%)
0,CASH,0.0,5.0615
1,NB,3.0,6.7300
2,EILB,6.0,12.8105
3,DMEQ,8.0,8.1964
4,EMEQ,11.0,19.9791
5,PE,12.0,16.4377
6,RE,5.0,9.1680


## Optimization

The data used for the optimization is the truncated and calibrated cube from the simulation step before. We have 2 optimizers, the `BaseOptimizer` and the `PortfolioOptimizer`. 

The `PortfolioOptimizer` is built on top of the `BaseOptimizer`. So if you need to do common optimization operations, use the `PortfolioOptimizer` as it probably already has the routines. If you need to work from scratch on something custom, use the `BaseOptimizer`.

We'll explore the `BaseOptimizer` first to understand how to code out a Maximize Expected Returns subject to CVaR and Volatility constraints.

$$ 
\underset{w_{1, \dots, A}}{\max} \frac{1}{N}\sum^N_i \prod^T_i \sum^A_k(\textbf{C}_{i, j, k} w_k + 1) \\
s.t. \\
\begin{align*}
\text{Vol}(\textbf{C}, \textbf{w}) &\leq 0.1 \\
\text{CVaR}(\textbf{C}, \textbf{w}) &\geq -0.33 \\
0 \leq w_k \leq 1 \quad &\forall \quad k \in 1, \dots, A
\end{align*}
$$

An important thing to note is that while we simulated monthly data, for optimization, we use quarterly data. Here's a fast way to convert monthly to quarterly data.

In [13]:
t, n, _ = cube.shape

q_cube = (cube + 1).reshape(t // 3, 3, n, num_assets).prod(1) - 1

Let's start with the `BaseOptimizer`

In [14]:
from allopy import BaseOptimizer

opt = BaseOptimizer(num_assets)

bounds = np.array([
    (0, 0.05),
    (0, 0.4),
    (0, 0.1),
    (0, 0.35),
    (0, 0.2),
    (0, 0.15),
    (0, 0.15),
])

opt.set_bounds(bounds[:, 0], bounds[:, 1])
max_vol = 0.1
max_cvar = -0.33

def obj_fun(w):
    # Maximize returns assuming there's rebalancing every quarter
    return ((q_cube @ w) + 1).prod(0).mean() - 1

def vol_c_fun(w):
    years = len(q_cube) // 4
    annual_data = (q_cube @ w + 1).reshape(years, 4, n).prod(1) - 1
    annual_vols = q_cube.std(0)
    return annual_vols.mean() - max_vol

def cvar_c_fun(w):
    # cvar usually calculated for 3 years
    returns = ((q_cube[:3 * 12] @ w) + 1).prod(0) - 1
    cutoff = np.percentile(returns, 5)  # get the 5%
    cvar = returns[returns <= cutoff].mean()
    
    return max_cvar - cvar


opt.add_equality_constraint(lambda w: sum(w) - 1)
opt.set_max_objective(obj_fun)
opt.add_inequality_constraint(vol_c_fun)
opt.add_inequality_constraint(cvar_c_fun)

weights = opt.optimize()

bopt = pd.DataFrame({
    'Asset': returns.columns,
    'Weights': np.round(weights * 100, 2)
})

bopt

,Asset,Weights
0,CASH,0.0
1,NB,5.0
2,EILB,10.0
3,DMEQ,35.0
4,EMEQ,20.0
5,PE,15.0
6,RE,15.0


Using the `PortfolioOptimizer`.

In [15]:
from allopy import PortfolioOptimizer, OptData

main_data = OptData(q_cube, time_unit='quarterly')
opt = PortfolioOptimizer(main_data)
opt.set_bounds(bounds[:, 0], bounds[:, 1])
weights = opt.PP.maximize_returns(max_vol=max_vol, max_cvar=max_cvar)

aopt = pd.DataFrame({
    'Asset': returns.columns,
    'Weights': np.round(weights * 100, 2)
})

aopt

,Asset,Weights
0,CASH,0.0
1,NB,5.0
2,EILB,10.0
3,DMEQ,35.0
4,EMEQ,20.0
5,PE,15.0
6,RE,15.0
